In [ ]:
%pip install lxml
%pip install requests

In [ ]:
### Trustpilot data scraping module 
### Author: Hakim Khalafi

## Imports

import math
import csv
import time
import json
import requests
import lxml.html as html

In [ ]:
## Configurations

# Trustpilot review page
basePage = 'http://www.trustpilot.com/review/'
reviewSite = 'www.skype.com'
reviewPage = basePage + reviewSite

# Data file to save to
datafile = 'dataSkype.csv'

# Trustpilot default 
resultsPerPage = 20 

print('Scraper set for ' + reviewPage + ' - saving result to ' + datafile)

In [ ]:
## Count amount of pages to scrape

# Get page, skipping HTTPS as it gives certificate errors
page = requests.get(reviewPage, verify=False)
tree = html.fromstring(page.content)

# Total amount of ratings
ratingCount = tree.xpath('//span[@class="headline__review-count"]')
ratingCount = int(ratingCount[0].text.replace(',',''))

# Amount of chunks to consider for displaying processing output 
# For ex. 10 means output progress for every 10th of the data
tot_chunks = 20

# Throttling to avoid spamming page with requests
# With sleepTime seconds between every page request
throttle = True
sleepTime = 1

# Total pages to scrape
pages = math.ceil(ratingCount / resultsPerPage)
print('Found total of ' + str(pages) + ' pages to scrape')

In [ ]:
## Main scraping section

with open(datafile, 'w', newline='', encoding='utf8') as csvfile:
    
    # Tab delimited to allow for special characters
    datawriter = csv.writer(csvfile, delimiter='\t')
    print('Processing..')
    for i in range(1,pages+1):
        
        # Sleep if throttle enabled
        if(throttle): time.sleep(sleepTime)

        page = requests.get(reviewPage + '?page=' + str(i))
        tree = html.fromstring(page.content)
        
        # Each item below scrapes a pages review titles, bodies and ratings
        script_bodies = tree.xpath("//script[starts-with(@data-initial-state, 'review-info')]")
        for idx,elem in enumerate(script_bodies):
            curr_item = json.loads(elem.text_content())

            # Progress counting, outputs for every processed chunk
            reviewNumber = idx + 20*(i-1) + 1
            chunk = int(ratingCount / tot_chunks)
            if(reviewNumber % chunk == 0): 
                print('Processed ' + str(reviewNumber) + '/'  + str(ratingCount) + ' ratings')
            
            title = curr_item["reviewHeader"]
            body = curr_item["reviewBody"]
            rating = curr_item["stars"]
            
            datawriter.writerow([title,body,rating])
            
    print('Processed ' + str(ratingCount) + '/' + str(ratingCount) + ' ratings.. Finished!')